In [5]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [6]:
productPrice = 20000
maxProductPrice = 25000
pricePerf = (maxProductPrice - productPrice) / maxProductPrice # Normalized price (0 is best, 1 is worst)

Information about customers

In [7]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [9]:
def getTotalUtilityForCustomer(car : COTSCar, c : str):
    total_utility = 0
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    total_utility += (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar]): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(car, c) for car in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [13]:
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = np.random.random() * maxProductPrice
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

designs = {car.name : car.vector for car in competitors}
prices = {car.name : car.price for car in competitors}
designs_df = pd.DataFrame(list(designs.items()), columns=['Name', 'Design'])
prices_df = pd.DataFrame(list(prices.items()), columns=['Name', 'Selling Price'])
competitorsDF = pd.merge(designs_df, prices_df, on='Name')
competitorsDF

,Name,Design,Selling Price
0,Competitor Car 0,"[2, 12, 10, 8, 1, 2, 5, 6, 15, 4, 24, 47, 3, 0...",24467.850519
1,Competitor Car 1,"[1, 4, 5, 1, 5, 3, 4, 20, 31, 2, 32, 34, 4, 2,...",19234.185339
2,Competitor Car 2,"[7, 5, 5, 7, 7, 0, 0, 11, 22, 1, 23, 102, 4, 2...",1462.411890


Create a random car and compute market share

In [14]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.price = np.random.random() * maxProductPrice
product.vector = carDesignVec
print("Our Car has design: ", product.vector, " and has selling price of ", product.price)

market_shares = {c : getMarketShare(product, c, competitors) for c in customers}
expected_customer_demand = {c : getMarketShare(product, c, competitors)*cQty[c] for c in customers}
market_shares_df = pd.DataFrame(list(market_shares.items()), columns=['Customer Type', 'MarketShare'])
customer_demand_df = pd.DataFrame(list(expected_customer_demand.items()), columns=['Customer Type', 'Expected Demand'])
df = pd.merge(market_shares_df, customer_demand_df, on='Customer Type')
df

Our Car has design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]  and has selling price of  3812.924247031252


,Customer Type,MarketShare,Expected Demand
0,CustomerType1,0.214214,44.342225
1,CustomerType2,0.114648,41.502734
2,CustomerType3,0.111155,6.224672
3,CustomerType4,0.168949,47.643680
4,CustomerType5,0.109135,67.882237
5,CustomerType6,0.124477,45.185021
6,CustomerType7,0.088636,87.306546
7,CustomerType8,0.149756,114.713341
8,CustomerType9,0.106067,12.303790
9,CustomerType10,0.166253,126.850718
